In [1]:
import random
import csv
from datetime import datetime, timedelta

# Users/roles in the system
users = ["student", "teacher", "admin"]
ips = ["192.168.1.5", "192.168.1.10", "10.0.0.25"]

# Normal query templates
normal_queries = [
    "SELECT * FROM students WHERE id={id};",
    "SELECT name, grade FROM grades WHERE student_id={id};",
    "INSERT INTO grades (student_id, course, grade) VALUES ({id}, 'CS101', 'A');",
    "UPDATE courses SET seats = seats - 1 WHERE course_id={id};",
    "DELETE FROM temp_data WHERE record_id={id};"
]

# Malicious query templates (attacks)
malicious_queries = [
    "SELECT * FROM users WHERE username='admin' OR '1'='1';",   # SQL Injection
    "SELECT * FROM students UNION SELECT card_number, ssn FROM payments;", # Data exfiltration
    "DROP TABLE students;",   # Privilege misuse
    "UPDATE users SET password='hacked' WHERE username='admin';", # Unauthorized update
    "SELECT * FROM secret_data;", # Unauthorized access
]

def generate_dataset(filename="synthetic_queries.csv", n=200):
    rows = []
    timestamp = datetime.now()

    for i in range(n):
        # Randomly decide normal (80%) or malicious (20%)
        if random.random() < 0.8:
            query = random.choice(normal_queries).format(id=random.randint(1, 500))
            label = 0
        else:
            query = random.choice(malicious_queries)
            label = 1

        row = {
            "timestamp": (timestamp + timedelta(seconds=i)).strftime("%Y-%m-%d %H:%M:%S"),
            "user": random.choice(users),
            "ip": random.choice(ips),
            "query": query,
            "label": label
        }
        rows.append(row)

    # Write to CSV
    with open(filename, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["timestamp", "user", "ip", "query", "label"])
        writer.writeheader()
        writer.writerows(rows)

    print(f"✅ Dataset generated: {filename} with {n} queries.")

# Run generator
if __name__ == "__main__":
    generate_dataset("synthetic_queries.csv", n=1000)


✅ Dataset generated: synthetic_queries.csv with 1000 queries.


In [2]:
import pandas as pd
df = pd.read_csv("synthetic_queries.csv")

In [3]:
df.head()

,timestamp,user,ip,query,label
0,2025-08-20 11:29:46,teacher,192.168.1.5,UPDATE courses SET seats = seats - 1 WHERE cou...,0
1,2025-08-20 11:29:47,admin,192.168.1.5,"SELECT name, grade FROM grades WHERE student_i...",0
2,2025-08-20 11:29:48,teacher,192.168.1.5,SELECT * FROM secret_data;,1
3,2025-08-20 11:29:49,student,192.168.1.5,UPDATE courses SET seats = seats - 1 WHERE cou...,0
4,2025-08-20 11:29:50,admin,10.0.0.25,"SELECT name, grade FROM grades WHERE student_i...",0


In [4]:
df['user'].iloc()

In [5]:
import numpy as np

def featurize(query, user):
    q = query.lower()
    return {
        "query_len": len(q),
        "num_digits": sum(c.isdigit() for c in q),
        "has_union": int("union" in q),
        "has_drop": int("drop" in q),
        "has_or1": int(" or 1=1" in q or " or '1'='1'" in q),
        "has_comment": int("--" in q or "/*" in q),
        "is_select": int(q.startswith("select")),
        "is_update": int(q.startswith("update")),
        "is_insert": int(q.startswith("insert")),
        "is_delete": int(q.startswith("delete")),
        "user": user  # now safe
    }

features = df.apply(lambda row: featurize(row['query'], row['user']), axis=1)
X = pd.DataFrame(list(features))
y = df['label']

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X['user'] = le.fit_transform(X['user'])

In [7]:
X

,query_len,num_digits,has_union,has_drop,has_or1,has_comment,is_select,is_update,is_insert,is_delete,user
0,57,4,0,0,0,0,0,1,0,0,2
1,51,2,0,0,0,0,1,0,0,0,0
2,26,0,0,0,0,0,1,0,0,0,2
3,57,4,0,0,0,0,0,1,0,0,1
4,51,2,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,35,2,0,0,0,0,1,0,0,0,0
996,52,3,0,0,0,0,1,0,0,0,1
997,57,4,0,0,0,0,0,1,0,0,0
998,58,0,0,0,0,0,0,1,0,0,1


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       160
           1       1.00      1.00      1.00        40

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [9]:
test_query = "SELECT * FROM users WHERE username='admin' OR '1'='1';"
x_new = pd.DataFrame([featurize(test_query)])
print("Prediction:", clf.predict(x_new))  # 1 = malicious


TypeError: featurize() missing 1 required positional argument: 'user'

In [ ]:
import joblib
joblib.dump(clf, "firstbatch.joblib")

['firstbatch.joblib']

In [ ]:
import psycopg2

conn = psycopg2.connect("dbname=securityComputing user=admin password=admin")
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER;")
user = cur.fetchone()[0]
if user == "admin":
    user = 2
elif user =="teacher":
    user = 1
else: user = 0
print("Query executed by:", user)
query = "SELECT * FROM students;"
# Run ML model here
x_new = pd.DataFrame([featurize(query, user)])
x_new
risk = clf.predict(x_new)[0]

if risk == 0:
    try:
        cur.execute(query)
    except Exception as e:
        print(f"Error {e}")
    print(cur.fetchall())
else:
    print("Query blocked 🚨")

Query executed by: 2


NameError: name 'clf' is not defined

In [ ]:
import joblib
import pandas as pd
import psycopg2

class Execute:
    def __init__(self,username,password):
        self.username = username
        self.password = password
        # self.category = category
        self.loaded_model = joblib.load("firstbatch.joblib")
        self.conn = None
        self.curr = None
        self.current_user = None
    def connection(self):
        try:
            
            self.conn = psycopg2.connect("dbname=securityComputing user={username} password={password}".format(username = self.username,password=self.password))
            self.cur = self.conn.cursor()
            self.cur.execute("SELECT CURRENT_USER;")
            self.current_user = self.cur.fetchone()[0]
            print("Query executed by:", self.current_user)
            role_map = {"student": 0, "teacher": 1, "admin": 2}
            self.current_user = role_map.get(self.current_user, -1)  # -1 if unknown
            
            print("Connection Established")
        except Exception as e:
            print(f"Error: {e}")

    def log_query(self, query, prediction, status):
        """Insert query log into DB"""
        try:
            log_query = """
                INSERT INTO query_logs (username, query, prediction, status)
                VALUES (%s, %s, %s, %s);
            """
            self.cur.execute(log_query, (self.current_user, query, int(prediction), status))
            self.conn.commit()
        except Exception as e:
            print(f"Logging error: {e}")
            self.conn.rollback()

    
    def query(self,query):
        try:
        # test_query = "SELECT * FROM users WHERE username='admin' OR '1'='1';"
            cleanedquery = pd.DataFrame([featurize(query,self.current_user)])
            # cleanedquery["has_user"] = self.current_user
            # print(cleanedquery)
            prediction = self.loaded_model.predict(cleanedquery)
        # print(prediction)
            if prediction[0] == 0:
                try:
                    self.cur.execute(query)
                    try:
                        rows = self.cur.fetchall()
                        print(rows)
                        # print(self.cur.fetchall())
                        self.log_query(query, prediction, "Executed")
                    except psycopg2.ProgrammingError:
                        print("Query executed successfully")
                        self.log_query(query, prediction, "Executed { no result}")
                    self.conn.commit()
                except Exception as e:
                    print(f"Execution Error: {e}")
                    self.log_query(query, prediction, f"Execution Error: {e}")
                # except psycopg2.ProgrammingError:   
                #     print(f"Query Executed ")
                # self.conn.commit()
            else:
                print("Query blocked 🚨")
                self.log_query(query,prediction,"Blocked")
        except Exception as e:
            print(f"Error: {e}")
            if self.conn:
                self.conn.rollback()
        


        
Test= Execute("admin","admin")
Test.connection()
Test.query("Select * from students where ID = 1")

Query executed by: admin
Connection Established
[(1, 'Alice Smith', 20, 'Computer Science')]


C:\Users\khatt\AppData\Local\Temp\ipykernel_20192\3853974496.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.cur.execute(log_query, (self.current_user, query, int(prediction), status))


In [10]:
    def query(self,query):
        try:
        # test_query = "SELECT * FROM users WHERE username='admin' OR '1'='1';"
            cleanedquery = pd.DataFrame([featurize(query)])
            prediction = self.loaded_model.predict(cleanedquery)
        # print(prediction)
            if prediction[0] == 0:
                self.cur.execute(query)
                try:
                    
                    print(self.cur.fetchall())


                except psycopg2.ProgrammingError:   
                    print(f"Query Executed ")
                self.conn.commit()
            else:
                print("Query blocked 🚨")
        except Exception as e:
            print(f"Error: {e}")
            if self.conn:
                self.conn.rollback()